In [4]:
import time
import csv
import re
import requests
import json
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("disable-gpu")

driver_path = 'D:/JISUNG/jupyter/chromedriver/chromedriver.exe'
browser = webdriver.Chrome(driver_path, chrome_options=options)
browser2 = webdriver.Chrome(driver_path, chrome_options=options)
browser.get('http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do')
count = int(browser.find_element_by_xpath('//*[@id="content"]/div[4]/div[1]/span/em').text.replace(',', ''))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/a[1]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="sPrdtStatStr"]'))
while 1:
    try:
        browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="mul_chk_det0"]'))
        break
    except:
        pass
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="layerConfirmChk"]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="searchForm"]/div[1]/div[5]/button[1]'))
while 1:
    try:
        browser.find_element_by_xpath('//*[@class="pageLoading"]')
    except:
        break
while 1:
    max_count = int(browser.find_element_by_xpath('//*[@id="content"]/div[4]/div[1]/span/em').text.replace(',', ''))
    if count != max_count:
        break
file = open('movie.csv', 'wt', encoding='utf-8', newline='')
writer = csv.writer(file)
writer.writerow(
    [
        'movie_id', # 영화 코드
        'title', # 제목
        'audience', # 관객수(종속변수)
        'release_date', # 개봉일
        'country', # 제작 국가
        'director', # 감독
        'producer', # 제작사
        'actor1', # 주연1
        'actor2', # 주연2
        'actor3', # 주연3
        'genre', # 장르
        'screen', # 스크린 수
        'running_time', # 상영시간
        'rating', # 등급
        'series', # 시리즈
        'holiday', # 개봉일부터 한 달 동안의 휴일 수
    ]
)

genres = ['드라마', '코미디', '액션', '멜로/로맨스', '스릴러', '미스터리',
          '공포(호러)', '어드벤처', '범죄', '가족', '판타지', 'SF',
          '서부극(웨스턴)', '사극', '애니메이션', '다큐멘터리', '전쟁',
          '뮤지컬', '성인물(에로)', '공연', '기타']

#########################################################
page_check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="pagingForm"]/div/a[1]'))
while 1:
    try:
        check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
        if page_check != check:
            break
    except:
        pass

for _ in range(225):
    page_check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="pagingForm"]/div/a[3]'))
    while 1:
        try:
            check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
            if page_check != check:
                break
        except:
            pass
        
page_check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@id="pagingForm"]/div/ul/li[7]/a'))
while 1:
    try:
        check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
        if page_check != check:
            break
    except:
        pass
#########################################################
        
page = 7
for i in range(22675, max_count + 1):
    try: 
        page_check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
        movie_id = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[3]/span').text.strip()
        title = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[1]/span/a').text.strip()
        temp = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[5]/span').text.strip()
        country = re.compile(r'[가-힣]+').search(temp).group()
        director = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[9]/span/a').text.strip()
        producer = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[10]/span/a').text.strip()
        temp = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[7]/span').text.strip()
        
        browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[1]/span/a'))
        while 1:                                  
            try:
                browser.find_element_by_xpath('/html/body/div[4]')
                break
            except:
                pass
        
        browser2.get('https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=')
        element = browser2.find_element_by_xpath('//*[@id="nx_query"]')
        element.send_keys(title)
        element.submit()
        try:
            series = ''
            browser2.find_element_by_xpath('//*[@id="_movie_tab"]/div/ul/li[4]/a')
            parent = browser2.find_element_by_xpath('//*[@id="_movie_tab_content_series"]/div[1]')
            child = parent.find_elements_by_tag_name('ul')
            for ul in child:
                for li in ul.find_elements_by_tag_name('li'):
                    for dl in li.find_elements_by_tag_name('dl'):
                        series_title = dl.find_element_by_xpath('./dt[1]/a').get_attribute('textContent')
                        if title != series_title:
                            series += series_title + '|'
        except NoSuchElementException:
            print(f'- {title}|시리즈 없음')
        except Exception as e:
            print(f'[ERROR] {i}번|{movie_id}|{title}\n{type(e)}\n{e}')
        
        if browser.find_element_by_xpath('//*[@class="ovf cont"]/dd[4]').text == '':
            temp = browser.find_element_by_xpath('//*[@class="ovf cont"]').get_attribute('textContent').replace(' ', '').replace('\n', '').replace('\t', '').split('|')
            for j in temp:
                if '분' in j:
                    running_time = j.split('분')[0]
                elif '관람' in j:
                    rating = j
                else:
                    for k in genres:
                        if k in j:
                            genre = j.replace(' ', '')
                            break
            
            release_date = re.compile(r'[0-9]{4}\-[0-9]{2}\-[0-9]{2}').search(temp[5]).group()
            
            try:
                temp = browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]').get_attribute('textContent').replace('배우[주연]', '').split('|')

                if len(temp) > 3:
                    for j in range(1, 4):
                        exec(f'actor{j} = temp[{j}].split()')
                else:
                    for j in range(1, len(temp) + 1):
                        exec(f'actor{j} = temp[{j}].split()')
                    for j in range(len(temp) + 1, 4):
                        exec(f'actor{j} = ""')
            except NoSuchElementException:
                print(f'- {title}|주연 정보 누락')
                actor1 = actor2 = actor3 = ''
            except Exception as e:
                print(f'[ERROR] {i}번|주연 정보 오류|{movie_id}|{title}\n{type(e)}\n{e}')

            browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="wrap_tab tab2"]/ul/li[2]'))
            while 1:
                try:
                    browser.find_element_by_xpath(f'//*[@class="item_tab statistics"]/p')
                except:
                    break

            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[2]').get_attribute('textContent')
                screen = re.compile(r'\d+\n').search(temp).group().strip()
            except NoSuchElementException:
                print(f'- {title}|스크린수 정보 누락')
                screen = ''
            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[4]').get_attribute('textContent')
                audience = re.compile(r'[\d]+\s').search(temp.replace(',', '')).group().strip()
            except NoSuchElementException:
                print(f'- {title}|관객수 정보 누락')
                audience = ''

        else:
            temp = browser.find_element_by_xpath('//*[@class="ovf cont"]/dd[4]').text.split('|')
            for j in temp:
                if '분' in j:
                    running_time = j.strip().split('분')[0]
                elif '관람' in j:
                    rating = j.strip()
                else:
                    for k in genres:
                        if k in j:
                            genre = j.replace(' ', '').replace(',', '|')
                            break
            
            if browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dt[5]').text.strip() == '개봉일':
                release_date = browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dd[5]').text.strip()
            else:
                release_date = browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dd[6]').text.strip()
            for j in range(1, 4):
                try:
                    if browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]/dt').text.strip() == '배우':
                        try:
                            temp = f'browser.find_element_by_xpath(\'//*[@class="staffMore"]/dl/div[2]/dd/table[1]/tbody/tr/td/a[{j}]\').text.strip()'
                            t = browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]/dd/table[1]/tbody/tr/td/a[{j}]').text.strip()
                            if re.compile(r"\(").search(t) == None:
                                exec(f'actor{j} = {temp}')
                            else:
                                exec(f'actor{j} = re.compile(r"[가-힣]+").search({temp}).group().strip()')
                        except NoSuchElementException:
                            exec(f'actor{j} = actor{j+1} = actor{j+2} = ""')
                            break
                        except Exception as e:
                            print(f'[ERROR] {i}번|주연 정보 오류|{movie_id}|{title}\n{type(e)}\n{e}')
                    else:
                        print(f'- {title}|주연 정보 누락')
                        actor1 = actor2 = actor3 = ''
                        break
                except NoSuchElementException:
                    print(f'- {title}|주연 정보 누락')
                    actor1 = actor2 = actor3 = ''
                    break
                except Exception as e:
                    print(f'[ERROR] {i}번|주연 정보 오류|{movie_id}|{title}\n{type(e)}\n{e}')

            browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="wrap_tab tab2"]/ul/li[2]'))
            while 1:
                try:
                    browser.find_element_by_xpath(f'//*[@class="item_tab statistics"]/p')
                except:
                    break

            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[2]').text
                screen = re.compile(r'\d+\n').search(temp).group().strip()
            except NoSuchElementException:
                print(f'- {title}|스크린수 정보 누락')
                screen = ''
            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[4]').text
                audience = re.compile(r'[\d]+\s').search(temp.replace(',', '')).group().strip()
            except NoSuchElementException:
                print(f'- {title}|관객수 정보 누락')
                audience = ''

        browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="hd_layer"]/a[2]'))

        holiday = 0
        if '없음' in release_date:
            release_date = ''
        else:
            temp = release_date.split('-')
            year = temp[0]
            month = temp[1]
            day = temp[2]
            date = datetime.date(int(year), int(month), int(day))


            for _ in range(31):
                d = (date).weekday()

                if date.year == int(year) and date.month == int(month):
                    url = f'https://m.search.naver.com/p/csearch/content/qapirender.nhn?where=nexearch&key=CalendarAnniversary&pkid=134&q={year}{month}%EC%9B%94&_callback=window.__jindo2_callback._directsearch_calendar_famous_month_0'
                else:
                    url = f'https://m.search.naver.com/p/csearch/content/qapirender.nhn?where=nexearch&key=CalendarAnniversary&pkid=134&q={date.year}{"0" + str(date.month) if len(str(date.month)) == 1 else str(date.month)}%EC%9B%94&_callback=window.__jindo2_callback._directsearch_calendar_famous_month_0'

                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'html.parser')
                data = re.compile(r'[(](.*?)[)]').search(str(soup)).group().replace('(', '').replace(')', '')
                parse = json.loads(data)

                day_off = 0
                for k in parse['openCalendar']['daysList']:
                    if k['solarDate'] == str(date).replace('-', ''):
                        if k['dayOff'] == 'true':
                            day_off = 1
                            break

                if d == 5 or d == 6 or day_off:
                    holiday += 1
                date += datetime.timedelta(days=1)
        
        print(f'{i}번 {movie_id}, {title}, {audience}, {release_date}, {country}, {director}, {producer}, {actor1}, {actor2}, {actor3}, {genre}, {screen}, {running_time}, {rating}, {series}, {holiday}')
        
        writer.writerow(
            [
                movie_id, 
                title,
                audience,
                release_date,
                country,
                director,
                producer,
                actor1,
                actor2,
                actor3,
                genre,
                screen,
                running_time,
                rating,
                series[:-1],
                holiday
            ]
        )
    except TimeoutException:
        webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform()
        print(f'[ERROR!] {i}번|{movie_id}|{title}\n{type(e)}\n{e}')
    except Exception as e:
        print(f'[ERROR!] {i}번|{movie_id}|{title}\n{type(e)}\n{e}')
    finally:
        print('')
        try:
            if not i % 10:
                if page % 10:
                    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@id="pagingForm"]/div/ul/li[{10 if page % 10 == 9 else (page + 1) % 10}]/a'))
                else:
                    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="pagingForm"]/div/a[3]'))
                page += 1
                while 1:
                    try:
                        check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
                        if page_check != check:
                            break
                    except:
                        pass
        except Exception as e:
            print(f'[ERROR!!] {i}번|{movie_id}|{title}\n{type(e)}\n{e}')
    
file.close()
browser.quit()
browser2.quit()

C:\Users\jejsm\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options
C:\Users\jejsm\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


- 진용|시리즈 없음
- 진용|스크린수 정보 누락
- 진용|관객수 정보 누락
22675번 19900350, 진용, , 1990-12-15, 홍콩, 정소동, , 공리, , , 사극|판타지, , 97, 중학생이상관람가, , 13

- 질투|시리즈 없음
- 질투|스크린수 정보 누락
- 질투|관객수 정보 누락
22676번 19830066, 질투, , 1983-07-02, 한국, 김인수, 신한영화(주), 고두심, 정윤희, 윤일봉, 드라마|멜로/로맨스, , 95, 연소자관람불가, , 10

- 집을 나온 여인|시리즈 없음
- 집을 나온 여인|스크린수 정보 누락
- 집을 나온 여인|관객수 정보 누락
22677번 19820013, 집을 나온 여인, , 1982-02-13, 한국, , 동아흥행㈜, 김지영, 윤일봉, 윤연경, 드라마|멜로/로맨스, , 110, 연소자관람불가, , 11

- 짚시애마|시리즈 없음
- 짚시애마|스크린수 정보 누락
- 짚시애마|관객수 정보 누락
22678번 19900081, 짚시애마, , 1990-09-01, 한국, , (주)두손필름, 후안 페드로, 이화란, , 멜로/로맨스|성인물(에로), , 110, 연소자관람불가, , 11

- 짧은 포옹 긴 이별|시리즈 없음
- 짧은 포옹 긴 이별|스크린수 정보 누락
- 짧은 포옹 긴 이별|관객수 정보 누락
22679번 19840028, 짧은 포옹 긴 이별, , 1984-04-05, 한국, 문여송, 현진필름, 박암, 최선아, 이효정, 드라마|멜로/로맨스, , 92, 국민학생관람불가, , 10

- 짬뽕 홍길동|스크린수 정보 누락
- 짬뽕 홍길동|관객수 정보 누락
22680번 19900051, 짬뽕 홍길동, , 1990-12-12, 한국, 임종호, (주)서울동화엔터프라이즈, 이창호, , , 멜로/로맨스|사극|액션|코미디, , 86, 연소자관람가, 슈퍼 홍길동 6 - 그림 도사와 홍길동,슈퍼 홍길동 5 - 부채 도사와 홍길동,슈퍼 홍길동 3,슈퍼 홍길동 2 - 공초 도사와 슈퍼 홍길동,슈퍼 홍길동,, 11

- 쫄병

- 친구여 조용히 가다오|시리즈 없음
- 친구여 조용히 가다오|스크린수 정보 누락
- 친구여 조용히 가다오|관객수 정보 누락
22728번 19820026, 친구여 조용히 가다오, , 1982-03-26, 한국, 고영남, 남아진흥(주), 독고성, 이영하, 원미경, 드라마, , 106, 국민학생관람불가, , 11

- 칠번통소사건|시리즈 없음
- 칠번통소사건|스크린수 정보 누락
- 칠번통소사건|관객수 정보 누락
22729번 19360009, 칠번통소사건, , 1936-11-15, 한국, 나운규, , 나운규, 박숙자, 임운학, 드라마|사극|액션, , 106, 국민학생관람불가, , 9

- 칠색조|시리즈 없음
- 칠색조|스크린수 정보 누락
- 칠색조|관객수 정보 누락
22730번 20090118, 칠색조, , 1990-10-20, 한국, 김인수, (주)하명중 영화제작소, 이상희, , , 드라마|멜로/로맨스, , 98, 연소자관람불가, , 10

- 칠소여복성|시리즈 없음
- 칠소여복성|스크린수 정보 누락
- 칠소여복성|관객수 정보 누락
22731번 19880018, 칠소여복성, , 1988-03-26, 한국, 강범구, 남아진흥, 조정현, 엽덕한, , 액션, , 95, 중학생이상관람가, , 11

- 칠지수|시리즈 없음
- 칠지수|스크린수 정보 누락
- 칠지수|관객수 정보 누락
22732번 19820030, 칠지수, , 1982-04-04, 한국, 남기남, 대양필름(주), 김국현, 한지하, 왕호, 사극|액션, , 80, 국민학생관람불가, , 10

- 침묵의 암살자|시리즈 없음
- 침묵의 암살자|주연 정보 누락
- 침묵의 암살자|스크린수 정보 누락
- 침묵의 암살자|관객수 정보 누락
22733번 19890318, 침묵의 암살자, , 1989-12-23, 미국, 이두용, , , , , 액션, , 90, 고등학생이상관람가, , 14

- 카멜레온의 시|시리즈 없음
- 카멜레온의 시|스크린수 정보 누락
- 카멜레온의 시|관객수 정보 누락
22734번 19880027, 카멜레

- 탱고와 캐쉬|시리즈 없음
- 탱고와 캐쉬|주연 정보 누락
22782번 19900295, 탱고와 캐쉬, 119, 1990-09-22, 미국, , , , , , 액션|코미디, 1, 100, 연소자관람불가, , 15

- 터미널|시리즈 없음
22783번 20040631, 터미널, 986903, 2004-08-27, 미국, 스티븐 스필버그, , 톰 행크스, 캐서린 제타-존스, , 드라마, 127, 128, 전체관람가, , 10

- 터프러버|시리즈 없음
- 터프러버|스크린수 정보 누락
- 터프러버|관객수 정보 누락
22784번 19890162, 터프러버, , 1989-05-19, 이탈리아, , , 란도 브잔카, , , 코미디, , 84, 연소자관람불가, , 11

- 테네시 벅크|시리즈 없음
- 테네시 벅크|스크린수 정보 누락
- 테네시 벅크|관객수 정보 누락
22785번 19900342, 테네시 벅크, , 1990-12-08, 미국, 데이빗 키스, , 데이빗 키스, , , 코미디, , 90, 중학생이상관람가, , 13

- 테크닉|시리즈 없음
- 테크닉|주연 정보 누락
- 테크닉|스크린수 정보 누락
- 테크닉|관객수 정보 누락
22786번 19890141, 테크닉, , 1989-04-12, 프랑스, , , , , , 드라마|코미디, , 81, 연소자관람불가, , 10

- 테크닉 2|시리즈 없음
- 테크닉 2|주연 정보 누락
- 테크닉 2|스크린수 정보 누락
- 테크닉 2|관객수 정보 누락
22787번 19900340, 테크닉 2, , 1990-12-01, 프랑스, , , , , , 드라마, , 92, 연소자관람불가, , 11

- 텐|시리즈 없음
- 텐|스크린수 정보 누락
- 텐|관객수 정보 누락
22788번 19820121, 텐, , 1982-12-25, 미국, , 오리온 픽쳐스 코퍼레이션, 더들리 무어, 로버트 웨버, 줄리 앤드류스, 드라마|코미디, , 118, 연소자관람불가, , 11

- 토스카니니|시리즈 없음
- 토스카니니|스크린수 정보 누락
- 토스

- 프란체스코|시리즈 없음
- 프란체스코|스크린수 정보 누락
- 프란체스코|관객수 정보 누락
22837번 19890269, 프란체스코, , 1989-09-30, 이탈리아, , , 미키 루크, , , 드라마, , 133, 연소자관람가, , 12

- 프레데터|주연 정보 누락
- 프레데터|스크린수 정보 누락
- 프레데터|관객수 정보 누락
22838번 19870115, 프레데터, , 1987-07-17, 미국, , , , , , SF|액션, , 109, 고등학생이상관람가, 프레데터스,프레데터 2,, 11

- 프레데터2|스크린수 정보 누락
- 프레데터2|관객수 정보 누락
22839번 19900358, 프레데터2, , 1990-12-22, 미국, 스티븐 홉킨스, , 루벤 블레디스, 대니 글로버, , SF|공포(호러)|스릴러|액션, , 106, 고등학생이상관람가, 더 프레데터,프레데터스,프레데터,, 13

- 프로그램 투 킬|시리즈 없음
- 프로그램 투 킬|스크린수 정보 누락
- 프로그램 투 킬|관객수 정보 누락
22840번 19880176, 프로그램 투 킬, , 1988-08-20, 미국, 알렌 홀그맨, , 로버트 기티, , , SF|액션, , 91, 고등학생이상관람가, , 10

- 프로펫서|시리즈 없음
- 프로펫서|스크린수 정보 누락
- 프로펫서|관객수 정보 누락
22841번 19890274, 프로펫서, , 1989-10-07, 이탈리아, , , 벤 가자라, , , 드라마|범죄, , 120, 고등학생이상관람가, , 11

- 프리찌스 오너|시리즈 없음
- 프리찌스 오너|스크린수 정보 누락
- 프리찌스 오너|관객수 정보 누락
22842번 19860116, 프리찌스 오너, , 1986-08-30, 미국, , , 리 리차드슨, 존 랜돌프, 안젤리카 휴스턴, 드라마|범죄|코미디, , 120, 연소자관람불가, , 13

- 프린세스 마담|스크린수 정보 누락
- 프린세스 마담|관객수 정보 누락
22843번 19890285, 프린세스 마담, , 1989-11-11, 홍콩, ,

- 호열자|시리즈 없음
- 호열자|스크린수 정보 누락
- 호열자|관객수 정보 누락
22892번 19200005, 호열자, , 1920-03-17, 한국, , , 하지만, , , 드라마, , 95, 연소자관람가, , 8

- 호호전|시리즈 없음
- 호호전|스크린수 정보 누락
- 호호전|관객수 정보 누락
22893번 19880074, 호호전, , 1988-12-03, 한국, 박호태, 성도흥업㈜, 이수진, 문태선, , 사극, , 103, 연소자관람불가, , 11

- 혼가|시리즈 없음
- 혼가|스크린수 정보 누락
- 혼가|관객수 정보 누락
22894번 19290006, 혼가, , 1929-01-26, 한국, 김유영, , 임화, 이영웅, , 드라마, , 103, 연소자관람불가, , 10

- 홀로서는 그날에|시리즈 없음
- 홀로서는 그날에|스크린수 정보 누락
- 홀로서는 그날에|관객수 정보 누락
22895번 19900007, 홀로서는 그날에, , 1990-04-21, 한국, 석래명, 한국영화배급㈜, 김정훈, 송승환, 하희라, 멜로/로맨스, , 108, 고등학생이상관람가, , 11

- 홍금보의 대나팔|시리즈 없음
- 홍금보의 대나팔|주연 정보 누락
- 홍금보의 대나팔|스크린수 정보 누락
- 홍금보의 대나팔|관객수 정보 누락
22896번 19870100, 홍금보의 대나팔, , 1987-05-02, 홍콩, 유관위, , , , , 액션|코미디, , 95, 고등학생이상관람가, , 11

- 홍금보의 방삼표|시리즈 없음
- 홍금보의 방삼표|주연 정보 누락
- 홍금보의 방삼표|스크린수 정보 누락
- 홍금보의 방삼표|관객수 정보 누락
22897번 19890101, 홍금보의 방삼표, , 1989-02-04, 홍콩, 장견정, , , , , 코미디, , 89, 중학생이상관람가, , 12

- 홍병매|시리즈 없음
- 홍병매|스크린수 정보 누락
- 홍병매|관객수 정보 누락
22898번 19840059, 홍병매, , 1984-11-03, 한국, 김시현, 화풍흥업, 서옥모, 장산, 김민경, 

22946번 19890328, 007 살인면허, , 1989-12-30, 미국, 존 글렌, 다냐크 프로덕션, , , , 드라마|범죄|스릴러|액션, , 132, 중학생이상관람가, , 14

- 007 제13탄 - 옥토퍼시|스크린수 정보 누락
- 007 제13탄 - 옥토퍼시|관객수 정보 누락
22947번 19840080, 007 제13탄 - 옥토퍼시, , 1984-07-29, 미국, , 다냐크 프로덕션, 로저 무어, 스티븐 버코프, 모드 아담스, 액션|어드벤처, , 130, 국민학생관람불가, , 10

- 007 제14탄 - 뷰 투 어 킬|스크린수 정보 누락
- 007 제14탄 - 뷰 투 어 킬|관객수 정보 누락
22948번 19850109, 007 제14탄 - 뷰 투 어 킬, , 1985-12-22, 미국, , 다냐크 프로덕션, 그레이스 존스, 패트릭 맥니, 타냐 로버츠, 액션, , 110, 국민학생관람불가, , 13

- 007 제15탄 - 리빙 데이라이트|주연 정보 누락
- 007 제15탄 - 리빙 데이라이트|스크린수 정보 누락
- 007 제15탄 - 리빙 데이라이트|관객수 정보 누락
22949번 19890107, 007 제15탄 - 리빙 데이라이트, , 1989-02-04, 미국, , 다냐크 프로덕션, , , , 액션, , 130, 중학생이상관람가, , 12

- 13월의 연정|시리즈 없음
- 13월의 연정|스크린수 정보 누락
- 13월의 연정|관객수 정보 누락
22950번 19820041, 13월의 연정, , 1982-06-05, 한국, , 태창엔터테인먼트(주), 박근형, 정한용, 이영옥, 드라마|멜로/로맨스, , 100, 연소자관람불가, , 10

- 13일의 금요일|주연 정보 누락
- 13일의 금요일|스크린수 정보 누락
- 13일의 금요일|관객수 정보 누락
22951번 19810066, 13일의 금요일, , 1981-08-08, 미국, 숀 S. 커닝햄, 파라마운트 픽쳐스, , , , 공포(호러)|미스터리|스릴러, , 95, 국민학생관람불가, 프레디 V

In [3]:
file.close()
browser.quit()
browser2.quit()